In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, bode, lsim, lti
from scipy import signal
from scipy.integrate import solve_ivp
from IPython.display import display, Latex, Markdown, Image, Math

## Kaskadenregelung

In [ ]:
imag = Image("../../images/zweitank.png", width=800)
display(imag)

## 2-Tanksystem

- lineares Modell im Zeitbereich
    \begin{align*}
        \dot{\tilde{z}}_1(t) & = a_{11} \tilde{z}_1(t) + b \tilde{u}_{\text{A}}(t)\\
        \dot{\tilde{z}}_2(t) & = a_{21} \tilde{z}_1(t) + a_{22} \tilde{z}_2(t) 
    \end{align*}
    mit Messungen $(y_1(t), y_2(t)) = (z_1(t),z_2(t))$
- lineares Modell im Bildbereich
    \begin{align*}
        P(s) &= \frac{\tilde{Z}_2(s)}{\tilde{U}_{\text{A}}(s)} = P_1(s) P_2(s)
    \end{align*}
    mit
    \begin{align*}
        P_1(s) & = \frac{K_1}{(1 + T_1 s)} = \frac{0.1059}{(1 + 68.21 s)}, &
        P_2(s) & = \frac{K_2}{(1 + T_2 s)} = \frac{2.7907}{(1 + 34.57 s)}, &
    \end{align*}
- PI-Regler für Folgeregler ($R_1$) zur Kompensation der Zeitkonstante von $P_1$
- PI-Regler für Führungsregler ($R_2$) für stationäre Genauigkeit

In [ ]:
imag = Image("../../images/tankKaskade.png", width=800)
display(imag)

__Definition Parammeter__

In [ ]:
tSim = np.linspace(0, 500, 501)

In [ ]:
x0SS = [0., 0.]
x0Kas = [0., 0., 0., 0.]
yRef = 0.2

In [ ]:
A = np.array([[-0.0289, 0], [0.0289, -0.0146]])
b = np.array([[0.0021], [0]])
c = np.array([[0], [1]])

In [ ]:
k = np.array([[31.6666666666666, 26.4814631735047]])
Kp1 = 29.5
Ki1 = 1 / 34.56729
Kp2 = 2
Ki2 = 1 / 18.75

## Simulation

In [ ]:
def prozessKaskade(t, x, yRef, A, b, Kp1, Kp2, Ki1, Ki2):
    e2 = yRef - x[1]
    u2 = Kp2 * e2 + x[3]
    
    e1 = u2 - x[0]
    u1 = np.array([Kp1 * e1 + x[2]])
    
    dx = np.zeros(4)
    dx[0:2] = A @ x[0:2] + b @ u1
    dx[2] = Ki1 * e1
    dx[3] = Ki2 * e2
    
    return dx

In [ ]:
def prozessVorfilter(t, x, yRef, A, b, c, k):
    V = 1 / (c.T @ np.linalg.inv(b @ k - A) @ b)
    u = V[0] * yRef - k @ x
    dx = A @ x + b @ u

    return dx

In [ ]:
resSS = solve_ivp(prozessVorfilter, [tSim[0], tSim[-1]], x0SS, t_eval=tSim, args=(yRef, A, b, c, k))

In [ ]:
resKas = solve_ivp(prozessKaskade, [tSim[0], tSim[-1]], x0Kas, t_eval=tSim, args=(yRef, A, b, Kp1, Kp2, Ki1, Ki2))

In [ ]:
plt.close()

fig, axes10 = plt.subplots(1, 1, figsize=(10,6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

axes10.plot(resSS.t, resSS.y[0], 'C0',label=r"Höhe $\tilde{z}_1$ - Zustand")
axes10.plot(resSS.t, resSS.y[1], 'C1', label=r"Höhe $\tilde{z}_2$ - Zustand")
axes10.plot(resKas.t, resKas.y[0], 'C0--', label=r"Höhe $\tilde{z}_1$- Kaskade")
axes10.plot(resKas.t, resKas.y[1], 'C1--', label=r"Höhe $\tilde{z}_2$- Kaskade")

axes11 = axes10.twinx()
axes11.plot(resSS.t, (k @ resSS.y).T, 'C4')
axes11.plot(resKas.t, Kp1 * ((Kp2 * (yRef - resKas.y[1]) + resKas.y[3]) - resKas.y[0]) + resKas.y[2], 'C4--')
axes11.set_ylabel(r"$\tilde{u}_{\mathrm{A}}$ in V", color='C4')
axes11.tick_params(axis='y', colors='C4')

axes10.set_xlabel(r"$t$ in s")
axes10.set_ylabel(r"$\tilde{z}_{\bullet}$ in m")

axes10.grid() 

handlesAx, labelsAx = axes10.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.90, 0.775, .15), loc=3,
           ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)
plt.show()